# Forest Stand Height Functions

In [ ]:
from numpy import *
import math as mt
import json
import scipy.io as sio
import subprocess
import os
import xml.etree.ElementTree as ET
import time
import argparse
import pdb
from osgeo import gdal, osr
import string
import pathlib


In [ ]:
def Crop_ISCE:
    xmlfile = "resampleOnlyImage.amp.xml"
    tree = ET.parse(xmlfile)
    root = tree.getroot()
    size_array = np.array([])
    for size in root.iter('property'):
        if size.items()[0][1] == 'size':
            size_array = np.append(size_array, int(size.find('value').text))
    width = size_array[0]
    length = size_array[1]
    
    nanval = 0
    
    # Read amp files in radar coordinates
    amp_file = np.fromfile("resampOnlyImage.amp", dtype = 'complex64')
    inty = amp_file.reshape((length, width))
    
    # Creating empty array for cropped square list
    
    inty[:176,:] = nanval
    inty[5488:,:] = nanval
    inty[:,:163] = nanval
    inty[:,4846:] = nanval
    
    # Write output files
    inty.tofile("resampOnlyImage.amp")

In [ ]:
def Crop_ROIPAC(directory, date1, date2):
    # Extract ROI_PAC parameters
    amp_rsc_file = date1 + "-" + date2 + "-" + "_2rlk.amp.rsc"
    width = int(rrd.read_rsc_data(amp_rsc_file, directory, "WIDTH"))
    length = int(rrd.read_rsc_data(amp_rsc_file, directory, "FILE_LENGTH"))
    fullwidth = width*2
    nanval = 0
    
    # Readcpr files in radar coordinates
    cor_file = np.fromfile(directory + date1 + "-" + date2 + "_2rlks.cor", dtype = "f4", count = length*fullwidth)
    corr = cor_file.reshape((length, fullwidth))
    mag = corr[:, 0:width]
    phs = corr[:.width:fullwidth]
    
    # Read amp files in radar coordinates
    amp_file = np.fromfile(directory + date1 + "-" + date2 + "_2rlks.amp", dtype = 'complex64')
    inty = amp_file.reshape((length, width))
    
    # Creating empty array for cropped square list
    mag[:638, :] = nanval
    mag[3288:, :] = nanval
    mag[:,:84] = nanval
    mag[:,2418:] = nanval
    
    phs[:638, :] = nanval
    phs[3288:, :] = nanval
    phs[:,:84] = nanval
    phs[:,2418:] = nanval
    
    inty[:638, :] = nanval
    inty[3288:, :] = nanval
    inty[:,:84] = nanval
    inty[:,2418:] = nanval
    
    # Writing values
    c_out[:, 0:width] = mag
    c_out[:,width:fullwidth] = phs
    
    # Write output files
    cx = c_out.astype('f4')
    cx.tofile(directory + date1 + "-" + date2 + "_2rlks_fix.cor")
    inty.tofile(directory + date1 + "-" + date2 + "_2rlks_fix.amp")

In [ ]:
def arc_sinc(x, C_param):
    # Get rid of extreme values by set all values where x > 1 equal to 1, and x < 0 equal to 0
    x[(x > 1)] = 1
    x[(x < 0)] = 0
    
    # Create array of increments between 0 and pi of size pi/100
    XX = linspace(0, math.pi, num = 100, endpoint = True)
    
    # Set the first value of XX to eps to avoid division by zero issues
    XX[0] = spacing(1)
    
    # Calculate sinc for for XX and save it to YY
    ## YY - sinc(XX / math.pi)
    YY = np.sin(XX) / XX
    
    # Reset the first value of XX to zero and the first value of YY to the corresponding output
    XX[0] = 0
    YY[0] = 1
    
    # Set the last value of YY to 0 to avoid NaN issues
    YY[-1] = 0
    
    # Flip XX and YY left to right
    YY = YY[::-1]
    XX = XX[::-1]
    
    # Run interpolation
    # XX and YY are your original values, x is the query value, and y is the interpolated values that correspond to x
    y = interp_func(x)
    
    # Set all values in y less than 0 equal to 0
    y[(y < 0)] = 0
    # return y
    return y

In [ ]:
# Input parameters are the numbers of scenes, edges, start scene, iterations, the input/output file directory, 
    # averaging numbers in lat and lon for "self" and "pairwise" fitting, bin_size for density calculation in scatter plot fitting, 
    # flag for sparse data cloud filtering.
def auto_mosaicking_new(scenes, edges, start_scene, N, linkarray, directory, Nd_pairwise, Nd_self, bin_size, flag_sparse):
    # Set average S and C parameters (0 < s < 1, 0 < c < 20 so s = 0.65, and c = 13)
    avg_S = 0.65
    avg_C = 13
    
    # Create avg_dp matric, and fill with average S and C parameters
    avg_dp = zeros(scenes * 2)
    put(avg_dp, range(0, scenes * 2, 2), avg_S)
    put(avg_dp, range(1, scenes * 2, 2), avg_C)
    
    # Create the dp matrix
    # the difference of the avg and the initial SC values OR all the zeros (avg)
    dp = zeros(scenes * 2)
    
    # Intialize target matrix nd fill with K = 1, B = 0
    target_KB = zeros((edges + 1) * 2)
    put(target_KB, range(0, (edges + 1) * 2, 2), 1)
    
    # Run cal_KB()
    Y = cKB.cal_KB(dp, edges, start_scene, linkarray, directory, Nd_pairwise, Nd_self, bin_size, flag_sparse)
    
    # Calculate the residual for cal_KB - target
    res = sum((Y - target_KB)**2)
    
    # Save dp and the residuals as the first iteration output file (using JSON)
    iter_file = open(os.path.join(dirextory, "output", "SC_0_iter.json"), 'w')
    json.dump([dp.tolist(), res], iter_file)
    iter_file.close()
    
    # For the rest of the iterations run ls_deltaSC() and save to output file (using JSON)
    for i in range(1, N + 1, 2): # This will run from i = 1 to i = N
        [dp, res] = lSC.ls_deltaSC(dp, edges, scenes, start_scene, linkarray, directory, Nd_pairwise, Nd_self, bin_size, flag_sparse)
        print("%d iterations completed!\n" % i)
        print(time.strftime("%H:%M:%S"))
        filename = "SC_%d_iter.json" % i
        iter_file = open(os.path.join(directory, "output", filename), 'w')
        json.dump([dp.tolist(), res], iter_file)
        iter_file.close()
        
    print("auto_mosaicking_new finsihed at " + (time.strftime("%H:%M:%S")))

In [ ]:
def auto_tree_height_many(scenes, flagfile, directory, numLooks, noiselevel, flag_proc, flag_grad):
    # for each scene name the file, run auto_tree_height_single and save the output to a .json file
    for i in range(scenes):
        
        # Get the scene data and set the file name and image folder name (f#_o# where # is the frame and orbit numbers, respectively)
        scene_data = fsf.flag_scene_file(flagfile, i + 1, directory) # 0 vs1 indexing
        filename = scene_data[1]
        image_folder = "f" + scene_data[4] + "_o" + scene_data[5] + "/"
        
        # Run auto_tree_height_single
        if flag_proc == 0:
            ######## ROI_PAC results
            file_date = athsR.auto_tree_height_single_ROIPAC(directory + image_folder, scene_data[2], scene_data[3], numLooks, noiselevel, flag_grad)
        elif flag_proc == 1:
            ######## ISCE results
            file_data = athsI.auto_tree_height_single_ISCE(directory + image_folder, scene_data[2], scene_data[3], numLooks, noiselevel, flag_grad)
        else:
            print("Invalid processor provided")
            
        linkfile = directory + image_folder + filename + '_orig.mat'
        
    sio.savemat(linkfile, {'corr_vs':file_data[0], 'ks':file_data[1], 'coords':file_data[2]})
    
    # Write geodata to a text file (4th - 9th values in file_data) 
    geofile = open(directory + image_folder + filename + "_geo.txt", "w")
    geofile.write("width: %d \n" % file_data[3])
    geofile.write("nlines: %d \n" % file_data[4])
    geofile.write("corner_lat: %f" % file_data[5])
    geofile.write("corner_lon: %f" % file_data[6])
    geofile.write("post_lat: %f" % file_data[7])
    geofile.write("post_lon: %f" % file_data[8])
    geofile.close()
    
    print ("auto_tree_height_many finished at " + (time.strftime("H:%M:%S")))

In [ ]:
def auto_tree_height_single_ISCE(directory, date1, date2, numLooks, noiselevel, flag_grad):
    # Extract ISCE parameters
    xmlfile = subprocess.getoutput('find ' + directory + 'int_' + date1 + '_' + date2 + '/ -name *Proc.xml')
    tree = ET.parse(xmlfile)
    root = tree.getroot()
    root_tag = root.tag
    
    range_pixel_res = float(root.findall("./master/instrument/range_pixel_size")[0].text)
    llambda = float(root.findall("./master/instrument/radar_wavelength")[0].text)
    try:
        first_range = float(root.findall("./runTopo/inputs/range_first_sample")[0].text)
    except:
        first_range = float(root.findall("./runTopo/inputs/RANGE_FIRST_SAMPLE")[0].text)
    try:
        num_range_bin = int(root.findall("./runTopo/inputs/width")[0].text)
    except:
        num_range_bin = int(root.findall("./runTopo/inputs/WIDTH")[0].text)
    try:
        num_range_looks = int(root.findall("./runTopo/inputs/number_range_looks")[0].text)
    except:
        num_range_looks = int(root.findall("./runTopo/inputs/NUMBER_RANGE_LOOKS")[0].text)
    center_range = first_range + (num_range_bin/2-1)*range_pixel_res*num_range_looks
    incid_angle = float(root.findall("./master/instrument/incidence_angle")[0].text)
    baseline_top = float(root.findall("./baseline/perp_baseline_top")[0].text)
    baseline_bottom = float(root.findall("./baseline/perp_baseline_bottom")[0].text)
    baseline = (baseline_bottom+baseline_top)/2
    

    xmlfile = directory+"int_"+date1+"_"+date2+"/topophase.cor.geo.xml"
    tree = ET.parse(xmlfile)
    root = tree.getroot()
    delta_array = np.array([])
    start_array = np.array([])
    size_array = np.array([], dtype=np.int32)
    for size in root.iter('property'):
        if size.items()[0][1] == 'size':
            size_array = np.append(size_array, int(size.find('value').text))
    for delta_val in root.iter('property'):
        if delta_val.items()[0][1] == 'delta':
            delta_array = np.append(delta_array, float(delta_val.find('value').text))
    for start_val in root.iter('property'):
        if start_val.items()[0][1] == 'startingvalue':
            start_array = np.append(start_array, float(start_val.find('value').text))
    end_array = start_array + size_array * delta_array
    north = max(start_array[1],end_array[1])
    south = min(start_array[1],end_array[1])
    east = max(start_array[0],end_array[0])
    west = min(start_array[0],end_array[0])
    coords = [north, south, west, east]
    geo_width = size_array[0]
    geo_nlines = size_array[1]
    corner_lat = north
    corner_lon = west
    step_lat = delta_array[1]
    step_lon = delta_array[0]

    xmlfile = directory+"int_"+date1+"_"+date2+"/resampOnlyImage.amp.geo.xml"
    tree = ET.parse(xmlfile)
    root = tree.getroot()
    delta_array = np.array([])
    start_array = np.array([])
    size_array = np.array([], dtype=np.int32)
    for size in root.iter('property'):
        if size.items()[0][1] == 'size':
            size_array = np.append(size_array, int(size.find('value').text))
    if (size_array[0]<geo_width)|(size_array[1]<geo_nlines):
        for delta_val in root.iter('property'):
            if delta_val.items()[0][1] == 'delta':
                delta_array = np.append(delta_array, float(delta_val.find('value').text))
        for start_val in root.iter('property'):
            if start_val.items()[0][1] == 'startingvalue':
                start_array = np.append(start_array, float(start_val.find('value').text))
        end_array = start_array + size_array * delta_array
        north = max(start_array[1],end_array[1])
        south = min(start_array[1],end_array[1])
        east = max(start_array[0],end_array[0])
        west = min(start_array[0],end_array[0])
        coords = [north, south, west, east]
        geo_width = size_array[0]
        geo_nlines = size_array[1]
        corner_lat = north
        corner_lon = west
        step_lat = delta_array[1]
        step_lon = delta_array[0]


    # Read geolocated amp and cor files

    fid_cor = open(directory + "int_"+date1+"_"+date2+"/topophase.cor.geo", "rb")
    cor_file = np.fromfile(fid_cor, dtype=np.dtype('<f'))

    corr = cor_file.reshape(2*geo_width, -1, order='F')
    corr = corr[:,0:geo_nlines]
    corr_mag = corr[geo_width:2*geo_width,:]

    fid_amp = open(directory + "int_"+date1+"_"+date2+"/resampOnlyImage.amp.geo", "rb")
    amp_file = np.fromfile(fid_amp, dtype=np.dtype('<f'))
    inty = amp_file.reshape(2*geo_width, -1, order='F')
    inty = inty[:,0:geo_nlines]
    inty1 = inty[::2,:]
    inty2 = inty[1::2,:]


    # Operations
    inty1 = np.power(inty1,2) # Hardcoded based on 2 range looks and 10 azimuth looks
    inty2 = np.power(inty2,2)

    inty1[inty1 <= 0] = np.NaN
    inty2[inty2 <= 0] = np.NaN
    corr_mag[corr_mag <= 0] = np.NaN

    ####### Noise level for ISCE-processed SAR backscatter power output
    if noiselevel == 0.0:
        if root_tag[0] == 'i':
            ####### ALOS thermal noise level (insarApp)
            N1 = 55.5**2
            N2 = 55.5**2
        elif root_tag[0] == 's':
            ####### ALOS thermal noise level (stripmapApp)
            N1 = (55.5/81)**2
            N2 = (55.5/81)**2
        else:
            raise Exception("invalid *Proc.xml file!!!")
    else:
        N1 = noiselevel
        N2 = noiselevel


    S1 = inty1 - N1
    g_th_1 = np.zeros(S1.shape)
    g_th_1[S1>N1] = np.sqrt(S1[S1>N1] / (S1[S1>N1] + N1))
    g_th_1[np.isnan(S1)] = np.NaN
    g_th_1[S1 <= N1] = np.NaN

    S2 = inty2-N2
    g_th_2 = np.zeros(S2.shape)
    g_th_2[S2>N2] = np.sqrt(S2[S2>N2] / (S2[S2>N2] + N2))
    g_th_2[np.isnan(S2)] = np.NaN
    g_th_2[S2 <= N2] = np.NaN

    g_th = g_th_1 * g_th_2

    corr_mag[corr_mag<0] = 0
    corr_mag[corr_mag>1] = 1
    corr_mag = rcb.remove_corr_bias(corr_mag,numLooks)
    corr_mag[corr_mag<0] = 0

    corr_vs = corr_mag / g_th

    # set constants
    pi=mt.pi

    # correcting geometric decorrelation related to value compensation of ROI result compared to GAMMA. Caused by baseline/other decorrelation
    gamma_base = 1 - (2 * mt.fabs(baseline) * mt.cos(incid_angle / 180 * pi) * range_pixel_res / mt.sin(incid_angle / 180 * pi) / llambda / center_range)
    gamma_geo = gamma_base
    corr_vs = corr_vs / gamma_geo
    corr_vs[corr_vs>1] = 1

    ##### Simple Radiometric correction of the coherences
    if flag_grad == 1:
        y = np.linspace(1, geo_width, geo_width)
        x = np.linspace(1, geo_nlines, geo_nlines)
        [X, Y] = np.meshgrid(x, y)
        A = np.vstack([X[~np.isnan(corr_vs)], Y[~np.isnan(corr_vs)], np.ones(np.size(corr_vs[~np.isnan(corr_vs)]))]).T
        coeff = np.linalg.lstsq(A, corr_vs[~np.isnan(corr_vs)])[0]
        corr_vs = corr_vs - X*coeff[0] - Y*coeff[1]
        corr_vs[corr_vs>1] = 1
        corr_vs[corr_vs<0] = 0

    kz = -2 * pi * 2 / llambda / center_range / mt.sin(incid_angle/180*pi) * baseline
    kz = mt.fabs(kz)

    # Return corr_vs, kz, coords
    return corr_vs, kz, coords, geo_width, geo_nlines, corner_lat, corner_lon, step_lat, step_lon

In [ ]:
def read_rsc_data(filename, directory, param):
    # set default output value
    result = -1
    
    # Set filenmae for file to be searched
    rsc_file = dirextory + filenmae
    
    # Read parameters from file
    for line in open(rsc_file):
        if line.startswith(param):
            result = float(line.strip().split()[1])
    
    return result

In [ ]:
def cal_KB(dp, edges, start_scene, link, directory, Nd_pairwise, Nd_self, bin_size .flag_sparse):
    
    # make output matric of zeros 
    YY = zeros((edges + 1) * 2)
    if link.size != 0:
        # for each edge run cal_KB_pairwise_new an put theoutput into YY
        fir i in range(edges):
            k_temp, b_temp = kbp.cal_KB_pairwise_new(int(link[i, 0]), int(link[i, 1]), dp[int((2*link[i, 0])-2)], dp[int((2*link[i, 0])-1)], dp[int((2*link[i, 1])-2)], dp[int((2*link[i, 1])-1)], directory, Nd_pairwise, bin_size)
            YY[2 * i] = k_temp
            YY[(2 * i)] = b_temp
            
        # run cal_KB_self_new and put output into YY
        
        k_temp, b_temp = kbs.cal_KB_self_new(dp[int((2 * start_scene) - 2)], dp[int((2 * start_scene) - 1)], directory, Nd_self, bin_size, flag_sparse)
        YY[(2 * (edges + 1)) - 2] = k_temp
        YY[(2 * (edges + 1)) - 1] = b_temp
        
        # Return Y
        return YY

In [ ]:
def cal_KB_pairwise_new(scene1, scene2, deltaS1, deltaC1, deltaS2, deltaC2, directory, Nd_pairwise, bin_size):
    # Set main file name string as scene1_scene2
    file_str == str(scene1) + '_' + str(scene2)
    
    # Load and read data from .mat file
    # Samples and lines are calculated from the shape of the images
    selffile_data = sio.loadmat(directory + "output/" + file_str + ".mat")
    image1 = selffile_data['I1']
    image2 = selffile_data['I2']
    lines = int(image1.shape[0])
    samples = int(image1.shape[1])
    
    # S and C parameters are the average S and C plus the delta value
    S_param1 = 0.65 + deltaS1
    S_param2 = 0.65 + deltaS2
    C_param1 = 13 + deltaC1
    C_param2 = 13 + deltaC2
    
    # Create gamma and ren arc_since for image1
    gamm1 = image1.copy()
    gamma1 = gamma1 / S_param1
    image1 = arc.arc_since(gamma1, C_param1)
    image1[isnan(gamma1)] = nan
    
    # Create gamma and run arc_since for image2
    gamma2 = image2.copy()
    gamma2 = gamma2 / S_param2
    image2 = arc.arc_since(gamma2, C_param2)
    image2[isnan(gamma2)] = nan
    
    # Partition image into subsections for noise suppression (multi-step process)
    # Create M and N which are the number of subsections in each direction; fix() rounds towards zero
    # NX and NY are the subsection dimensions
    NX = Nd_pairwise
    NY = Nd_pairwise
    M = int(fix(lines / NY))
    N = int(fix(samples / NX))
    
    # Create JM and JN which is the remainder adter dividing into subsections
    JM = lines % NY
    JN = samples % NX
    
    # Select the portion of images that are within the subsections
    image1 = image1[0:lines - JM][:, 0:samples - JN]
    image2 = image2[0:lines - JM][:, 0:samples - JN]
    
    # Split each image into subsections and run mean_wo_nan on each subsection
    
    # Declare new arrays to hold the subsection averages
    image1_means = zeros((M, N))
    image2_means = zeros((M, N))
    
    # Processing image1
    # Split image into subsections with NY of rows in each
    
    image1_rows = split(image1, M, 0)
    for i in range(M):
        # Split each section into subsections with NX number of columns in each
        row_array_split(image1_rows[i], N, 1)
        # for each subsection shape take the mean with NaN and save the value in another array
        for j in range(N):
            image1_means[i, j] = mwn.mean_wo_nan(row_array[j])
            
    # Processing image2
    # split image into subsections with NY number of rows in each
    image2_rows = split(image2, M, 0)
    for i in range(M):
        # split each section into subsections with NX number of columns ineach
        row_array = split(image2_rows[i], N, 1)
        # for each subsection shape take the mean without NaN and save the value in another array
        for j in range(N):
            image2_means[i, j] = mwn.mean_wo_nan(row_array[j])
            
    # Make an array for each image of where mean > 0 for both images
    IND1 = logical_and((image1_means > 0), (image2_means > 0))
    I1m_trunc = image1_means[IND1, ...]
    I2m_trunc = image2_means[IND1, ...]
    
    I1m_trunc, I2m_trunc = rout.remove_outlier(I1m_trunc, I2m_trunc, 0.5, 2)
    
    # Extract density values from the 2D scatter plot
    
    I1m_den = I1m_trunc
    I2m_den = I2m_trunc
    
    # Calculate the covariance matrix of the data with outliers removed
    cov_matrix = cov(I1m_den, I2m_den)
    
    # Calculate the eigenvalues
    dA, vA = linalg.eig(cov_matrix)
    
    # Calculate K and B 
    # K is based on whichever value is dA is the largest
    if (dA[0] > dA[1]): # dA[0] is largest
        K = vA[1, 0] / vA[0, 0]
    else: # dA[1] is largest
        K = vA[1, 1] / vA[0, 1]
    B = 2 * mean(I1m_den - I2m_den) / mean(I1m_den + I2m_den)
    
    return K, B

In [ ]:
def cal_KB_self_new(deltaS2, deltaC2, directory, Nd_self, bin_size, sparse_lidar_flag):
    selffile_data = sio.loadmat(o.spath.join(directory, "output", "self.mat"))
    image1 = selffile_data['I1']
    image2 = selffile_data['I2']
    lines = int(image1.shape[0])
    samples = int(image1.shape[1])
    
    # Set the S and C parameters to the average S and C plus the delta values
    S_param2 = 0.65 + deltaS2
    C_param2 = 13 + deltaC2
    
    # Create gamma and run arc_since for image2
    gamma2 - image2.copy()
    gamma2 = gamma2 / S_param2
    image2 = arc.arc_sinc(gamma2, C_param2)
    image2[inan(gamma2)] = nan
    
    # Partition image into subsections for noise suppression (multi-step process)
    # Create M and N which are the number of subsections in each direction
    # NC and NY are the subsection dimensions
    NX = Nd_self
    NY = Nd_self
    M = int(fix(lines / NY))
    N = int(fix(samples/ NX))
    
    # Create JM and JN which is te remainder after dividing into subsections
    JM = lines % NY
    JN = samples % NX
    
    # Select the portions of images that are within the subsections
    image1 = image1[0:lines - JM][:, 0:samples - JN]
    image2 = image2[0:lines - JM][:, 0:samples - JN]
    
    # Split each image into subsections and run mean_wo_nan on each subsection
    
    # Declare new array to hold subsections averages
    image1_means = zeros((M, N))
    image2_means = zeros((M, N))
    
    # Processing image1
    # Split image into sections with NY number or rows each
    image1_rows = split(image1, M, 0)
    for i in range(M):
        # Split each section into subsections with NX number of columns in each
        row_array = split(image1_rows[i], N, 1)
        # for each subsection shape take the mean wihout NaN and save the value in another array
        for j in range(N):
            image1_means[i, j] = mwn.mean_wo_nan(row_array[j])
            
    # Processing image2
    # Split image into subsections with NY number of rows in each
    image2_rows = split(image2, M, 0)
    for i in range(M):
        # Split each section into subsections with NX number of columns
        row_array = split(image1_rows[i], N, 1)
        # for each subsection shape take the mean without NaN and save the value in another array
        for j in range(N):
            image2_means[i, j] = mwn.mean_wo_nan(row_array[j])
    
    # Make an array for each image of where mean > 0 for both images
    IND1 = logical_and((image1_means > 0), (immage2_means > 0))
    I1m_trunc = image1_means[IND1, ...]
    I2m_trunc = image2_means[IND1, ...]
    
    # Remove the overestimate at low height end (usually subject to imperfection of the mask
    # over water bodies, farmlands and human activities) and the saturation point over the forested areas due to logging
    IND2 - logical_or((I1m_trunc < 5), (I2m_trunc > (mt.pi * C_param2 - 1)))
    IND2 = logical_not(IND2);
    
    
    # Call remote_outlier on these cells when there are only a few of lidarsamples that are sparsely distributed
    if sparse_lidar_flag == 1:
        I1m_trunc = I1m_trunc[IND2, ...]
        I2m_trunc = I2m_trunc[IND2, ...]
        # Extract density values from the 2D scatter plot
        I1m_den, I2m_denespd.extract_scatterplot_density(I1m_trunc, I2m_trunc, bin_size)
    else:
        I1m_trunc, I2m_trunc = rout.remove_outlier(I1m_trunc, I2m_trunc, 0.5, 2)
        I1m_den = I1m_trunc
        I2m_den = I2m_trunc
    
    # Calculate the covariance matrix of the data with outliers removed
    cov_matrix = cov(I1m_den, I2m_den)
    
    # Calculate the eigenvalues
    dA, vA = linalg.eig(cov_matrix)
    
    # Calculate K and B
    # K is based on whichever value in dA is the largest
    if (dA[0] > dA[1]): # dA[0] is largest
        K = vA[1,0] /vA[0,0]
    else: # dA[1] is largest
        K = vA[1, 1] /vA[0, 1]
    B = 2 * mean(I1m_den - I2m_den) / mean(I1m_den + I2m_den)
    
    return K, B

In [ ]:
def cal_error_metric(dp, edges, start_scene, link, directory, N_pairwise, N_self):
    # make output matrix of zeros
    YY = zeros((edges + 1) * 2)
    if link.size != 0:
        # for each edge run cal_error_metric_pairwise and put the output into YY
        for i in range(edges):
            R_temp, RMSE_temp = emp.cal_error_metric_pairwise(int(link[i, 0]), int(link[i, 1]), dp[int((2*link[i, 0])-2)], dp[int((2*link[i, 0])-1)], dp[int((2*link[i, 1])-2)], dp[int((2*link[i, 1])-1)], directory, N_pairwise)
            YY[2 * i] = R_temp
            YY[(2 * i) + 1] = RMSE_temp 
            
    # Run cal_error_metric_self and put output into YY
    R_temp, RMSE_temp = ems.cal_error_metric_self(dp[int((2 * start_scene) - 2)], dp[int((2 * start_scene) - 1)], directory, N_self)
    YY[(2 * (edges + 1)) - 2] = R_temp
    YY[(2 * (edges + 1)) - 1] = RMSE_temp
    
    # return Y
    return YY

In [ ]:
def cal_error_metric_pairwise(scene1, scen2, deltaS1, deltaC1, deltaS2, deltaC2, directory, N_pairwise):
    # Set main file name string as scene1_scene2
    file_str = str(scene1) + '_' + str(scene2)
    
    # Load and read data from .mat file
    # Samples and lines are calculated from the shape of the images
    selffile_data = sio.loadmat(os.path.join(directory, "output", file_str + ".mat"))
    image1 = selffile_data['I1']
    image2 = selffile_data['I2']
    lines = int(image1.shape[0])
    samples = int(image1.shape[1])
    
    # S and C parameters are the average S and C plus the delta value
    S_param1 = 0.65 + deltaS1
    S_param2 = 0.65 + deltaS2
    C_param1 = 13 + deltaC1
    C_param2 = 13 + deltaC2
    
    # Create gamma and run arc_since for image1
    gamma1 = image1.copy()
    gamma1 = gamma1 / S_param1
    image1 = arc.arc_sinc(gamma1, C_param1)
    image1[isnan(gamma1)] = nan
    
    # Create gamma and run arc_since for image2
    gamma2 = image2.copy()
    gamma2 = gamma2 / S_param2
    image2 = arc.arc_sinc(gamma2, C_param2)
    image2[isnan(gamma2)] = nan
    
    # Partition image into subsections for noise suppression (multi-step process)
    # Create M and N which are the number of subsections in each direction; fix() rounds towards zero
    # NX and NY are the subsection dimensions
    NX = N_pairwise
    NY = N_pairwise
    M = int(fix(lines / NY))
    N = int(fix(samples / NX))
    
    # Create JM and JN, which is the remainder after dividing into subsections
    JM = lines % NY
    JN = samples % NX
    
    # Select the portions of images that are within the subsections
    image1 = image1[0:lines - JM][:, 0:samples - JN]
    image2 = image2[0:lines - JM][:, 0:samples - JN]
    
    # Split each image into subsections and run mean_wo_nan on each subsection
    
    # Declare new arrays to hold the subsection averages
    image1_means = zeros((M, N))
    image2_means = zeros((M, N))
    
    # Processing image1
    # Split image into subsections with NY number of rows in each
    image1_rows = split(image1, M, 0)
    for i in range(M):
        # split each section into subsections with NX number of columns in each
        row_array = split(image1_rows[i], N, 1)
        # for each subsection shape take the mean without NaN andsave the value in another array
        for j in range(N):
            image1_means[i, j] = mwn.mean_wo_nan(row_array[j])
    
    # Processing image2
    # Split image into sections with NY number of rows in each
    image2_rows = split(image2, M, 0)
    for i in range(M):
        # split each section into subsections with NX number of columns in each
        row_array = split(image2_rows[i], N, 1)
        # for each subsection shape take the mean without NaN an dsave the values in another array
        for j in range(N):
            image2_means[i, j] = mwn.mean_wo_nan(row_array[j])
    
    # Make an array for each image of where mean > 0 for both images
    IND1 = logical_and((image1_means > 0), (image2_means > 0))
    I1m_trunc = image1_means[IND1, ...]
    I2m_trunc = iamge2_means[IND1, ...]
    
    R = corrcoef(I1m_trunc, I2m_trunc)
    R = R[0,1]
    RMSE = sqrt(sum((I1m_trunc - I2m_trunc)**2)/I1m_trunc.size)
    
    # Export the pair of heights for future scatter plot
    filename = file_str + "_I1andI2.json"
    R_RMSE_file = open(os.path.join(directory, "output", filename), 'w')
    json.dump([I1m_trunc.tolist(), I2m_trunc.tolist()], R_RMSE_file)
    R_RMSE_file.close()
    
    return R, RMSE

In [ ]:
def cal_error_metric_self(deltaS2, deltaC2, directory, N_self):
    # load and read data from .mat file
    # Samples and lines are calculated from the shape of the images
    selffile_data = sio.loadmat(os.path.join(directory, "output", "self.mat"))
    image1 = selffile_data['I1']
    image2 = selffile_data['I2']
    lines = int(image1.shape[0])
    samples = int(image1.shape[1])
    
    # set the S and C parameters to the average S and C plus the delta values
    S_param2 = 0.65 + deltaS2
    C_param2 = 13 + deltaC2
    
    # Create gamma and run arc_since for image2
    gamma2 = image2.copy()
    gamma2 = gamm2 / S_param2
    image2 = arc.arc_sinc(gamma2, C_param2)
    image2[isnan(gamma2)] = nan
    
    # Partition image into subsections for noise suppression (multi-step process)
    # Create M and N which are the number of subsections in each direction
    # NX and NY aer the subsection dimensions 
    NX = N_self
    NY = N_eslf
    M = int(fix(lines / NY))
    N = int(fix(samples / NX))
    
    # Create JM and JN, which is the remainder after dividing into subsections
    JM = lines % NY
    JN = samples % NX
    
    # Select the portions of images that are within the subsections
    image1 = image1[0:lines - JM][:, 0:samples - JN]
    image2 = image2[0:lines - JM][:, 0:samlpes - JN]
    
    # Split each image into subsections and run mean_wo_nan on each subsection
    
    # Declare new array to hold subsection averages
    image1_means = zeros((M, N))
    image2_means = zeros((M, N))
    
    # Processing image1
    # Split image into sections with NY number of rows in each
    image1_rows = split(image1, M, 0)
    for i in range(M):
        # split each section subsections with NX number of columns in each 
        row_array = split(iamge_rows[i], N, 1)
        # for each subsection shape take the mean without NaN and save the value in another array
        for j in range(N):
            image1_means[i, j] = mwn.mean_wo_nan(row_array[j])
            
    # Processing image2
    # Split image into sections with NY number of rows in each
    image2_rows = split(image2_rows[i], N, 1)
    for i in range(M):
        # split each section into subsections with NX number o fcolumns in each
        for j in range(N):
            image2_means[i, j] = mwn.mean_wo_nan(row_array[j])
    
    # Make an array for each imaeg where mean > 0 for both images
    IND1 = logical_and((image1_means > 0), (image2_means > 0))
    I1m_trunc = image1_means[IND1, ...]
    I2m_trunc = image2_means[IND1, ...]
    
    R = corrcoef(I1m_trunc, I2m_trunc)
    R = R[0, 1]
    RMSE = sqrt(sum((I1m_trunc-I2m_trunc)**2)/I1m_trunc.size)
    
    filename = "self_I1andI2.json"
    R_RMSE_file = open(os.path.join(directory, "output", filename), 'w')
    json.dump([I1m_trunc.tolist(), I2m_trunc.tolist()], R_RMSE_file)
    R_RMSE_file.close()
    
    return R, RMSE

In [ ]:
def create_mosaic(directory, mosaicfile): 
    
    print(time.strftime("%H:%M:%S"))
    
    # Making list of files
    listofffile = ''
    iterdir = [f for f in os.listdir('.') if os.path.isdir(f) and f.startswith('f')]
    
    for num, val in enumerate(iterdir):
        os.chdir(os.path.join(directory, val))
        tiffile = [f for f in os.listdir('.') if f.endswith('fsh.tif')]
        abspth = str(pathlib.Path(tiffile).absolute())
        listoffiles = listoffiles + ' ' + abspth
    
    os.chdir(directory)
    print(directory)
    subprocess.getoutput('gdalbuildvrt -seperate -srcnodata 255 -overwrite ' + os.path.join(directory, 'mosaic.vrt') + listoffiles)
    subprocess.getoutput('gdal_translate -of GTiff -a_nodata 255 ' + os.path.join(directory, 'mosaic.vrt') + ' ' + os.path.join(dirextory, 'mosaic.tif'))
    
    # Load mosaic.tif and associated parameters - .tif
    driver = gdal.GetDriverByName('GTiff')
    drier.Register()
    img = gdal.Open(os.path.join(directory, 'mosaic.tif'))
    ref_data = array(img.ReadAsArray())
    refgeotrans = img.GetGeoTransform()
    corner_lon = refgeotrans[0]
    post_lon = refgeotrans[1]
    corner_lat = refgeotrans[3]
    post_lat = refgeotrans[5]
    geo_width = img.RasterXSize
    geo_lines = img.RasterYSize
    
    ######### average all of the overlappingpixels at the same area
    ref_data = single(ref_data)
    ref_data[ref_data==255] = NaN
    avg = nanmean(ref_data, axis = 0)
    avg[isnan(avg)] = 255
    
    ######### Create the final GeoTiff
    driver = gdal.GetDriverByName('GTiff')
    
    outRaster = driver.Create(os.path.join(directory, mosaicfile), geo_width, geo_lines)
    outRaster.SetGeoTransform([corner_lon, post_lon, 0, corner_lat, 0, post_lat])
    outband = outRaster.GetRasterBand(1)
    
    outband.WriteArray(avg)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(4326)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()
    
    print(time.strftime("%H:%M:%S"))
    print(("Final mosaic generation done!!!"))
    
parser = argparse.ArgumentParser(description = "Create final mosaic map of forest stand height")
parser.add_argument('directory', type = str, help = 'the same root directory as forest_stand_height.py executes')
